<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/ResNet(cifar100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q wandb

     |████████████████████████████████| 1.8 MB 14.8 MB/s 
     |████████████████████████████████| 133 kB 57.8 MB/s 
     |████████████████████████████████| 170 kB 66.8 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 138 kB 60.9 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 62 kB 927 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

from wandb.keras import WandbCallback
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
cifar100 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
def relu_bn(x):
    x = ReLU()(x)
    x = BatchNormalization()(x)
    return x

def residual_block(x, downsample, filters, kernel_size=3):
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    x = BatchNormalization()(inputs)
    x = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(x)
    x = relu_bn(x)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            x = residual_block(x, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    x = AveragePooling2D(4)(x)
    x = Flatten()(x)
    outputs = Dense(100, activation='softmax')(x)
    
    model = Model(inputs, outputs)

    return model

In [ ]:
def experiment():
    config_defaults = {
        'epochs': 2,
        'batch_size': 1024,
        'lr': 0.1
    }

    wandb.init(project='VGGs', config=config_defaults, magic=True)
    args = wandb.config
    
    # Prepare dataloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # init
    keras.backend.clear_session()
    
    net = create_res_net()
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    net.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    result = net.fit(
        trainloader, 
        epochs=args.epochs, 
        validation_data=valloader, 
        batch_size=args.batch_size,
        callbacks=[WandbCallback(save_model=False)])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [ ]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'lr': {
            'values': [0.01, 0.001, 0.0001]
        },
        'epochs': {
            'values': [25]
        },
        'batch_size': {
            'values': [1024]
        }
    }
}

import wandb
sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=experiment)

Create sweep with ID: km23mv7v
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/km23mv7v


wandb: Agent Starting Run: 8aiho63z with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 25
wandb: 	lr: 0.01
wandb: Currently logged in as: jeongmin (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Ignored wandb.init() arg project when running a sweep


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/25
40/40 [==============================] - 7650s 191s/step - loss: 3.0427 - accuracy: 0.1470 - val_loss: 19556513792.0000 - val_accuracy: 0.0970
Epoch 2/25
16/40 [===========>..................] - ETA: 1:14:08 - loss: 2.1975 - accuracy: 0.2053

[Experiment](https://wandb.ai/jeongmin/uncategorized/sweeps/ef7doh12?workspace=user-jeongmin)